In [1]:
import math
import numpy as np
import random
import rbo
import matplotlib.pyplot as plt
from collections import Counter
import pandas as pd
import dataframe_image as dfi
from scipy.stats import binom, norm, uniform


/var/folders/08/2z4j5f7s6p79tdhfg6p3grdw0000gn/T/ipykernel_27342/1517185123.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Domain Generation

In [2]:
def jaccard_similarity(set1, set2):
    intersection = set1.intersection(set2)
    union = set1.union(set2)

    return len(intersection) / len(union)

In [3]:
def generate_domains(n, jac_value):
    # first, we find the size of the intersection based on the given jaccard_value
    intersection_size = math.floor((jac_value * (2 * n)) / (1 + jac_value))

    set_a = set()
    set_b = set()
    
    for i in range(intersection_size):
        item_label = 'i' + str(i)
        set_a.add(item_label)
        set_b.add(item_label)

    # remaining number of elements
    
    a_rem = n - intersection_size
    b_rem = n - intersection_size

    for i in range(a_rem):
        set_a.add('a' + str(i))

    for i in range(b_rem):
        set_b.add('b' + str(i))

    

    return set_a, set_b

## Default Overlap Distributions

In [20]:
def exponential_decay(theta):
    # theta determines the level of top-weightedness.
    def f(depth, n):    
        d = depth - 1 # depth is 1 indexed while the function is 0 indexed
    
        k = (0.3 * n) / np.log(n)
        
        exp_term = np.exp(-1 * (d / k))
    
        linear_term = d / n

        return (theta * exp_term) + ((1 - theta) * uniform_term)
        
    return f


def logarithmic_growth():
    # theta determines the level of top-weightedness.
    def f(depth, n):    

        return (np.log(depth) / np.log(n+1))
        
    return f


def gaussian_distribution(mean, std_dev):
    def f(depth, n):
        d = depth
        gaussian = norm(loc=mean, scale=std_dev)

        scale = gaussian.pdf(mean)

        return gaussian.pdf(d) / scale
    return f

    

def scaled_binomial_probability_function(p):
    def f(depth, n):
        largest_probability_x = n * p
        scale_factor = 1/binom.pmf(largest_probability_x, n, p)
        return (scale_factor * binom.pmf(depth, n, p))
        
    return f
    

## Ties and Ranking Simulation

In [5]:
def ensure_two_separation(xs, n):

    xs.sort()

    altered_list = False

    for i in range(1, len(xs)):
        if (xs[i] - xs[i-1]) < 2 or (xs[i] == n-1):
            altered_list = True
            new = (xs[i] + 1) % n
            xs[i] = new

    # check again
    if altered_list:
        xs = ensure_two_separation(xs, n)
    
    return xs
        

def add_ties(X, frac_ties, num_groups, probabilities):

    if frac_ties == 0:
        return X

    assert num_groups <= len(X) / 2
    assert len(probabilities) == len(X)
    # assert np.sum(probabilities) == 1

    indices = np.arange(0, len(X))


    selected_start_indices = np.random.choice(indices, size=num_groups, replace=False, p=probabilities)
    selected_start_indices = ensure_two_separation(selected_start_indices, len(X))

    print(selected_start_indices)


    
    n = len(X)
    X_with_ties = []
    num_tied_items = math.floor(frac_ties * n)
    if (num_tied_items / 2) < num_groups:
        raise Error("Not enough groups")
        
    average_group_size = math.floor(num_tied_items / num_groups)
    
    i = 0
    
    actual_num_tied_items = 0
    actual_num_groups = 0

    
    while i < n:        
        if i in selected_start_indices:
            tie_group_length = np.random.poisson(average_group_size - 2) + 2
    
            tie_group = [X[i]]
            
            for j in range(1, tie_group_length):
                if ((i + j) in selected_start_indices):
                    i -= 1
                    break
                if i + j < n:
                    tie_group.append(X[i + j])
                    actual_num_tied_items += 1
    
            X_with_ties.append(tie_group)
            actual_num_groups += 1
            i += j + 1
        else:
            X_with_ties.append(X[i])
            i += 1

    return X_with_ties
    
    

In [6]:
def simulate_rankings(n, len_x, len_y, overlap_probability_function, tie_probabilities_x=None, tie_probabilities_y=None,
                      frac_ties_x=0, n_groups_x=0, frac_ties_y=0, n_groups_y=0, conjointness=1, return_truncated=True):
    '''
    - the overlap function is a discrete probability function over the number of items in the domain. It takes as input the current depth d
    '''
    
    # generate the two domains depending on the degree of conjointness
    a, b = generate_domains(n, conjointness)

    S = []
    L = []
    case = []
    agree_probs = []

    cases = [1, 2, 3, 4]

    decision = 0

    
    for depth in range(1, n+1):
        # sample randomly from domains a and b without replacement

        # agree_prob = agreement_probability(depth = depth, theta = theta, n = n)
        agree_prob = overlap_probability_function(depth = depth, n = n) # possibly allow for the option of adding more parameters

        agree_probs.append(agree_prob)

        u = np.random.uniform()

        item_S_domain = None
        item_L_domain = None

        item_S = None
        item_L = None

        if u < agree_prob:
            # CASE 1: 1/4 of the time, choose some element that is in the intersection of S domain (a) and L so far
            # CASE 2: 1/4 of the time, choose some element that is in the intersection of L domain (b) and S so far
            # CASE 3: 1/4 of the time, choose some element that has not yet been taken - from the intersection a and b
            # CASE 4: 1/4 of the time, choose some element that from L and add to S and vice versa.
       
            decision = random.sample(cases, 1)[0]            
            
            if decision == 1:
                item_S_domain = a.intersection(set(L))
                item_L_domain = b

            elif decision == 2:
                item_S_domain = a
                item_L_domain = b.intersection(set(S))
                
            elif decision == 3:
                item_S_domain = a.intersection(b)
                item_L_domain = a.intersection(b)

                # already draw item for S and L to make sure they are the same

                if len(item_S_domain) > 0:
                    item_S = random.sample(sorted(item_S_domain), 1)[0]
                    item_L = item_S
                    
                
            elif decision == 4:
                item_S_domain = a.intersection(set(L))
                item_L_domain = b.intersection(set(S))

            else:
                raise Error("Invalid decision")
            

        else:
            item_S_domain = a
            item_L_domain = b
            decision = 0


        if item_S is None:
            if len(item_S_domain) > 0:
                item_S = random.sample(sorted(item_S_domain), 1)[0]
            else:
                item_S = random.sample(sorted(a), 1)[0]
                decision = 0

        if item_L is None:
            if len(item_L_domain) > 0:
                item_L = random.sample(sorted(item_L_domain), 1)[0]
            else:
                item_L = random.sample(sorted(b), 1)[0]
                decision = 0

        case.append(decision)


        S.append(item_S)
        L.append(item_L)

        a.remove(item_S)
        b.remove(item_L)

    S_with_ties = add_ties(S, frac_ties_x, n_groups_x, probabilities=tie_probabilities_x)
    L_with_ties = add_ties(L, frac_ties_y, n_groups_y, probabilities=tie_probabilities_y)
    
    # after rankings have been made, truncate

    if return_truncated:
        return S_with_ties[:len_x], L_with_ties[:len_y], agree_probs[:len_x], case[:len_x]

    return S_with_ties, L_with_ties, agree_probs, case

## Format Ranking Dataframe

In [7]:
def get_formatted_dataframe(S, L, agreement_probabilities, case_list):
    data = {
            'S' : S,
            'L' : L,
            'overlap_probability' : agreement_probabilities,
            'case' : case_list
    }
    
    def highlight_equal_values(x):
        color = 'background-color: green'
        default = ''
        case1 = x['case'] == 1
        case2 = x['case'] == 2
        case3 = x['case'] == 3
        case4 = x['case'] == 4 
    
        if case1:
            return ['background-color: red','background-color: white', 'background-color: white', 'background-color: white']
        if case2:
            return ['background-color: white','background-color: purple', 'background-color: white', 'background-color: white']
        if case3:
            return ['background-color: #00A6D6','background-color: #00A6D6', 'background-color: white', 'background-color: white']
        if case4:
            return ['background-color: red','background-color: purple', 'background-color: white', 'background-color: white']
        else:
            return ['background-color: white'] * len(x)
    
    
    df = pd.DataFrame(data, index =  np.arange(1, np.max([len(S), len(L)]) + 1))
    df.index.name ='Rank'
    
    return df.style.apply(highlight_equal_values, axis=1)

In [8]:
def get_tied_dataframe(S, L, agreement_probability, case_list):

    S_flat = []
    L_flat = []
    S_colours = []
    L_colours = []

    def generate_random_colour():
        r = random.randint(0, 255)
        g = random.randint(0, 255)
        b = random.randint(0, 255)
        return f'#{r:02x}{g:02x}{b:02x}'
        

    for s in S:
        if isinstance(s, list) and len(s)>1:
            colour = generate_random_colour()
            for tied_item in s:
                S_flat.append(tied_item)
                S_colours.append(colour)
        else:
            S_flat.append(s)
            S_colours.append("white")

    for l in L:
        if isinstance(l, list) and len(l)>1:
            colour = generate_random_colour()
            for tied_item in l:
                L_flat.append(tied_item)
                L_colours.append(colour)
        else:
            L_flat.append(l)
            L_colours.append("white")
    
    data = {
            'S' : S_flat,
            'S_colour' : S_colours,
            'L' : L_flat,
            'L_colour' : L_colours,
            'overlap_probability' : agreement_probability,
            'case' : case_list
    }

    def colour_tied_items(x):
        return ['background-color: ' + x['S_colour'], 'background-color: white', 'background-color: ' + x['L_colour'], 'background-color: white', 'background-color: white', 'background-color: white']
        

    df = pd.DataFrame(data, index =  np.arange(1, np.max([len(S_flat), len(L_flat)]) + 1))
    df.index.name ='$d$'

    show = ['S', 'L', 'overlap_probability', 'case']


    styled_df = df.style.apply(colour_tied_items, axis=1)
    styled_df.hide([col for col in df.columns if col not in show], axis=1)
    return styled_df

In [9]:
def get_basic_dataframe(S, L):
    data = {
            'S' : S,
            'L' : L,
    }

    if len(L) < len(S):
        while len(L) < len(S):
            L.append(None)

    if len(S) < len(L):
        while len(S) < len(L):
            S.append(None)

    df = pd.DataFrame(data, index =  np.arange(1, np.max([len(S), len(L)]) + 1))
    df.index.name ='$d$'
    return df

# Examples

In [19]:
n = 2000

prob_dist_x = norm(loc=(0.75 * n), scale=n/4)
prob_dist_y = norm(loc=(0.25 * n), scale=n/16)

depths = np.arange(1, n+1)
probabilities_x = prob_dist_x.pdf(depths)
probabilities_y = prob_dist_y.pdf(depths)
    
probabilities_x = np.abs(probabilities_x)
probabilities_x /= probabilities_x.sum()

probabilities_y = prob_dist_y.pdf(depths)
    
probabilities_y = np.abs(probabilities_y)
probabilities_y /= probabilities_y.sum()



S, L, agreement_probability, case_list = simulate_rankings(n=n, len_x = 0, len_y = 0, overlap_probability_function=exponential_decay(theta=1), 
                                                           tie_probabilities_x=probabilities_x, tie_probabilities_y=probabilities_y,conjointness=1,                                               
                                                 return_truncated=False, frac_ties_x=0.6, n_groups_x=5, frac_ties_y=0.6, n_groups_y=5)
get_tied_dataframe(S, L, agreement_probability, case_list)

[ 716 1188 1648 1703 1974]
[225 435 597 629 674]


,S,L,overlap_probability,case
$d$,,,,
1,i187,i1538,1.000000,0
2,i1311,i187,0.987412,2
3,i197,i1311,0.974982,2
4,i1538,i141,0.962709,1
5,i141,i1609,0.950590,1
6,i1781,i1781,0.938623,3
7,i472,i197,0.926808,2
8,i815,i815,0.915141,3
9,i1525,i1525,0.903621,3


In [11]:
S, L, agree_probs, case_list = simulate_rankings(n=500, len_x = 500, len_y = 500, overlap_probability_function=logarithmic_growth(), conjointness=1,
                                                return_truncated=False)


get_formatted_dataframe(S, L, agree_probs, case_list)

,S,L,overlap_probability,case
Rank,,,,
1,i3,i358,0.000000,0
2,i399,i452,0.111499,0
3,i315,i481,0.176722,0
4,i430,i156,0.222999,0
5,i82,i272,0.258893,0
6,i20,i20,0.288221,3
7,i391,i343,0.313018,0
8,i358,i430,0.334498,4
9,i176,i176,0.353444,0


In [12]:
S, L, agree_probs, case_list = simulate_rankings(n=500, len_x = 500, len_y = 500, overlap_probability_function=scaled_binomial_probability_function(0.5), conjointness=1,
                                                return_truncated=False)


get_formatted_dataframe(S, L, agree_probs, case_list)

,S,L,overlap_probability,case
Rank,,,,
1,i235,i320,0.000000,0
2,i279,i103,0.000000,0
3,i426,i335,0.000000,0
4,i395,i218,0.000000,0
5,i167,i99,0.000000,0
6,i33,i284,0.000000,0
7,i466,i315,0.000000,0
8,i114,i222,0.000000,0
9,i368,i381,0.000000,0


Save image for the poster

In [13]:
S, L, agree_probs, case_list = simulate_rankings(n=500, len_x = 500, len_y = 500, overlap_probability_function=gaussian_distribution(120, 10), conjointness=1,
                                                return_truncated=False)


get_formatted_dataframe(S, L, agree_probs, case_list)

,S,L,overlap_probability,case
Rank,,,,
1,i410,i86,0.000000,0
2,i17,i65,0.000000,0
3,i61,i300,0.000000,0
4,i251,i248,0.000000,0
5,i385,i172,0.000000,0
6,i455,i98,0.000000,0
7,i494,i192,0.000000,0
8,i237,i361,0.000000,0
9,i79,i290,0.000000,0


In [14]:
# dfi.export(styled_df, 't0.5c1.png')

In [15]:
S, L, agree_probs, case_list = simulate_rankings(n=10, len_x = 0, len_y = 0, overlap_probability_function=scaled_binomial_probability_function(0.6), conjointness=1,
                                                return_truncated=False)
get_formatted_dataframe(S, L, agree_probs, case_list)

,S,L,overlap_probability,case
Rank,,,,
1,i6,i2,0.006271,0
2,i5,i6,0.042328,0
3,i2,i1,0.169312,0
4,i4,i0,0.444444,0
5,i7,i5,0.800000,0
6,i9,i9,1.000000,3
7,i1,i7,0.857143,1
8,i0,i8,0.482143,0
9,i3,i4,0.160714,0


Find the average RBO scores

In [16]:
# theta_values = [0, 0.2, 0.4, 0.6, 0.8, 1.0]

# average_rbo_ext = []

# for theta in theta_values:
#     rbo_exts = []
#     for _ in range(100):
#         S, L, _, _ = simulate_rankings(n=200, len_x = 60, len_y = 60, overlap_probability_function=agreement_probability(theta=theta), conjointness=1)
#         # S, L, _ = simulate_rankings(n=200, len_x = 60, len_y = 60, conjointness=1, theta=theta)
#         rbo_ext = rbo.RankingSimilarity(S, L).rbo_ext(p=0.95)
#         rbo_exts.append(rbo_ext)
#     average_rbo_ext.append(np.mean(rbo_exts))


In [17]:
# rbo_ext_df = pd.DataFrame(
#     {
        
#         '$RBO_\text{ext}$' : average_rbo_ext
#     },
#     index = theta_values
# )

# rbo_ext_df.index.name = "$\\theta$"
# rbo_ext_df.T

In [18]:
# dfi.export(rbo_ext_df, 'rbo_ext_df.png')